In [6]:
import requests
import pandas as pd
import json
import numpy as np
from typing import Dict, List, Tuple
from datetime import datetime, timedelta
import time

# ========== НАСТРОЙКИ ==========
PRICE_IMPACT_PCT = 1.0          # Price impact в процентах (±1%)
MIN_LIQUIDITY_USD = 1000        # Минимальная ликвидность для фильтрации ($)
ORDERBOOK_LIMIT = 200           # Количество уровней в стакане
REQUEST_DELAY = 0.1             # Задержка между запросами (сек)
TOP_CONTRACTS_DISPLAY = 20      # Количество контрактов для отображения
TOP_LIQUIDITY_DISPLAY = 5       # Топ контрактов по ликвидности
LIQUIDITY_DEPTH_LEVELS = 50     # Уровни для расчета глубины ликвидности

# Настройки арбитража
CAPITAL = 50000                 # Капитал в USDT
LEVERAGE = 5                    # Плечо
CURRENT_DATE = datetime(2025, 7, 1)  # Текущая дата для расчетов
# ================================

class BTCETHArbitrageAnalyzer:
    def __init__(self):
        self.base_url = "https://api.bybit.com"
        self.session = requests.Session()
        self.position_size = CAPITAL / 2  # Делим капитал пополам для лонг и шорт позиций
        self.leveraged_position = self.position_size * LEVERAGE
        
    def get_contracts_by_asset(self, base_coin: str) -> List[Dict]:
        """Получить все контракты для определенного актива"""
        try:
            url = f"{self.base_url}/v5/market/instruments-info"
            params = {
                'category': 'linear',
                'baseCoin': base_coin
            }
            
            response = self.session.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if data['retCode'] == 0:
                return data['result']['list']
            else:
                print(f"Ошибка API для {base_coin}: {data['retMsg']}")
                return []
                
        except Exception as e:
            print(f"Ошибка при получении контрактов {base_coin}: {e}")
            return []
    
    def get_orderbook(self, symbol: str, limit: int = ORDERBOOK_LIMIT) -> Dict:
        """Получить стакан заявок для символа"""
        try:
            url = f"{self.base_url}/v5/market/orderbook"
            params = {
                'category': 'linear',
                'symbol': symbol,
                'limit': limit
            }
            
            response = self.session.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if data['retCode'] == 0:
                return data['result']
            else:
                print(f"Ошибка получения стакана для {symbol}: {data['retMsg']}")
                return {}
                
        except Exception as e:
            print(f"Ошибка при получении стакана {symbol}: {e}")
            return {}
    
    def get_ticker(self, symbol: str) -> Dict:
        """Получить текущую цену символа"""
        try:
            url = f"{self.base_url}/v5/market/tickers"
            params = {
                'category': 'linear',
                'symbol': symbol
            }
            
            response = self.session.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if data['retCode'] == 0 and data['result']['list']:
                return data['result']['list'][0]
            else:
                return {}
                
        except Exception as e:
            print(f"Ошибка при получении тикера {symbol}: {e}")
            return {}
    
    def filter_contracts(self, contracts: List[Dict], asset: str) -> List[Dict]:
        """Фильтрация контрактов по активу"""
        filtered_contracts = []
        
        for contract in contracts:
            symbol = contract['symbol']
            contract_type = contract.get('contractType', '')
            
            # Исключаем Inverse контракты
            if contract_type in ['InverseFutures', 'InversePerpetual']:
                continue
            
            # Логика фильтрации по активам
            if asset == 'BTC':
                # Исключаем старые форматы
                if any(excluded in symbol for excluded in ['BTCPERP', 'BTC-26SEP25', 'BTC-26DEC25']):
                    continue
                
                # Оставляем BTCUSDT (Perpetual) и BTCUSDT-DDMMMYY (Futures)
                if symbol == 'BTCUSDT' and contract_type == 'LinearPerpetual':
                    filtered_contracts.append(contract)
                elif symbol.startswith('BTCUSDT-') and contract_type == 'LinearFutures':
                    if len(symbol.split('-')) == 2 and len(symbol.split('-')[1]) == 7:
                        filtered_contracts.append(contract)
            
            elif asset == 'ETH':
                # Оставляем ETHUSDT (Perpetual) и ETHUSDT-DDMMMYY (Futures)
                if symbol == 'ETHUSDT' and contract_type == 'LinearPerpetual':
                    filtered_contracts.append(contract)
                elif symbol.startswith('ETHUSDT-') and contract_type == 'LinearFutures':
                    if len(symbol.split('-')) == 2 and len(symbol.split('-')[1]) == 7:
                        filtered_contracts.append(contract)
        
        return filtered_contracts
    
    def calculate_liquidity_metrics(self, orderbook: Dict, current_price: float, 
                                  price_impact_pct: float = PRICE_IMPACT_PCT) -> Dict:
        """Рассчитать метрики ликвидности с учетом price impact"""
        
        if not orderbook or 'b' not in orderbook or 'a' not in orderbook:
            return {
                'max_buy_volume_usdt': 0,
                'max_sell_volume_usdt': 0,
                'buy_liquidity_depth': 0,
                'sell_liquidity_depth': 0,
                'spread_pct': 0,
                'mid_price': current_price
            }
        
        bids = [[float(bid[0]), float(bid[1])] for bid in orderbook['b']]
        asks = [[float(ask[0]), float(ask[1])] for ask in orderbook['a']]
        
        if not bids or not asks:
            return {
                'max_buy_volume_usdt': 0,
                'max_sell_volume_usdt': 0,
                'buy_liquidity_depth': 0,
                'sell_liquidity_depth': 0,
                'spread_pct': 0,
                'mid_price': current_price
            }
        
        best_bid = bids[0][0]
        best_ask = asks[0][0]
        mid_price = (best_bid + best_ask) / 2
        spread_pct = ((best_ask - best_bid) / mid_price) * 100
        
        # Рассчитываем пороговые цены с учетом price impact
        buy_threshold_price = current_price * (1 + price_impact_pct / 100)
        sell_threshold_price = current_price * (1 - price_impact_pct / 100)
        
        # Считаем максимальный объем покупки
        max_buy_volume_usdt = 0
        for ask_price, ask_size in asks:
            if ask_price <= buy_threshold_price:
                max_buy_volume_usdt += ask_price * ask_size
            else:
                remaining_price_room = buy_threshold_price - ask_price
                if remaining_price_room > 0:
                    partial_volume = (remaining_price_room / ask_price) * ask_size * ask_price
                    max_buy_volume_usdt += partial_volume
                break
        
        # Считаем максимальный объем продажи
        max_sell_volume_usdt = 0
        for bid_price, bid_size in bids:
            if bid_price >= sell_threshold_price:
                max_sell_volume_usdt += bid_price * bid_size
            else:
                remaining_price_room = bid_price - sell_threshold_price
                if remaining_price_room > 0:
                    partial_volume = (remaining_price_room / bid_price) * bid_size * bid_price
                    max_sell_volume_usdt += partial_volume
                break
        
        # Общая глубина ликвидности в стакане
        buy_liquidity_depth = sum([price * size for price, size in asks[:LIQUIDITY_DEPTH_LEVELS]])
        sell_liquidity_depth = sum([price * size for price, size in bids[:LIQUIDITY_DEPTH_LEVELS]])
        
        return {
            'max_buy_volume_usdt': max_buy_volume_usdt,
            'max_sell_volume_usdt': max_sell_volume_usdt,
            'buy_liquidity_depth': buy_liquidity_depth,
            'sell_liquidity_depth': sell_liquidity_depth,
            'spread_pct': spread_pct,
            'mid_price': mid_price,
            'best_bid': best_bid,
            'best_ask': best_ask,
            'buy_threshold_price': buy_threshold_price,
            'sell_threshold_price': sell_threshold_price
        }
    
    def parse_expiry_date(self, symbol: str, asset: str) -> datetime:
        """Парсинг даты экспирации из названия контракта"""
        perpetual_symbols = {
            'BTC': 'BTCUSDT',
            'ETH': 'ETHUSDT'
        }
        
        if symbol == perpetual_symbols.get(asset):
            return None  # Perpetual контракт
        
        try:
            # Формат: BTCUSDT-04JUL25, ETHUSDT-04JUL25, SOLUSDT-04JUL25
            date_part = symbol.split('-')[1]  # 04JUL25
            day = int(date_part[:2])          # 04
            month_str = date_part[2:5]        # JUL
            year = int('20' + date_part[5:])  # 2025
            
            month_map = {
                'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
            }
            
            if month_str in month_map:
                return datetime(year, month_map[month_str], day)
            
        except:
            pass
        
        return None
    
    def calculate_arbitrage_profit(self, perpetual_price: float, futures_price: float, 
                                 expiry_date: datetime) -> Dict:
        """Расчет прибыли от арбитража"""
        if expiry_date is None:
            return {}
        
        days_to_expiry = (expiry_date - CURRENT_DATE).days
        if days_to_expiry <= 0:
            return {}
        
        # Расчет прибыли от арбитража
        price_difference = futures_price - perpetual_price
        
        # Количество монет в позиции
        asset_amount = self.leveraged_position / perpetual_price
        
        # Прибыль в USDT (разница цен * количество монет)
        profit_usdt = price_difference * asset_amount
        
        # Прибыль в процентах от капитала
        profit_percentage = (profit_usdt / CAPITAL) * 100
        
        # Годовая доходность (экстраполяция)
        annual_return = (profit_percentage / days_to_expiry) * 365
        
        return {
            'price_difference': price_difference,
            'days_to_expiry': days_to_expiry,
            'asset_amount': asset_amount,
            'profit_usdt': profit_usdt,
            'profit_percentage': profit_percentage,
            'annual_return': annual_return,
            'expiry_date': expiry_date
        }
    
    def analyze_asset_contracts(self, asset: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Анализ контрактов для конкретного актива"""
        print(f"\n--- Анализируем {asset} контракты ---")
        
        contracts = self.get_contracts_by_asset(asset)
        if not contracts:
            return pd.DataFrame(), pd.DataFrame()
        
        filtered_contracts = self.filter_contracts(contracts, asset)
        print(f"Найдено {len(contracts)} {asset} контрактов, отфильтровано {len(filtered_contracts)}")
        
        liquidity_results = []
        arbitrage_results = []
        perpetual_price = None
        
        # Определяем символ perpetual контракта для актива
        perpetual_symbols = {
            'BTC': 'BTCUSDT',
            'ETH': 'ETHUSDT'
        }
        perpetual_symbol = perpetual_symbols.get(asset)
        
        # Сначала получаем цену perpetual контракта
        if perpetual_symbol:
            print(f"Получаем цену perpetual контракта {perpetual_symbol}...")
            perpetual_ticker = self.get_ticker(perpetual_symbol)
            if perpetual_ticker:
                perpetual_price = float(perpetual_ticker.get('lastPrice', 0))
                if perpetual_price > 0:
                    print(f"Цена {asset} perpetual ({perpetual_symbol}): ${perpetual_price:.2f}")
                else:
                    print(f"Не удалось получить цену для {perpetual_symbol}")
                    return pd.DataFrame(), pd.DataFrame()
            else:
                print(f"Не удалось получить тикер для {perpetual_symbol}")
                return pd.DataFrame(), pd.DataFrame()
        
        # Анализируем все контракты
        for i, contract in enumerate(filtered_contracts):
            symbol = contract['symbol']
            print(f"Анализируем {symbol} ({i+1}/{len(filtered_contracts)})...")
            
            # Получаем текущую цену
            ticker = self.get_ticker(symbol)
            if not ticker:
                continue
                
            current_price = float(ticker.get('lastPrice', 0))
            if current_price == 0:
                continue
            
            # Получаем стакан заявок
            orderbook = self.get_orderbook(symbol)
            
            # Рассчитываем метрики ликвидности
            liquidity_metrics = self.calculate_liquidity_metrics(orderbook, current_price)
            
            # Добавляем информацию о ликвидности
            liquidity_result = {
                'asset': asset,
                'symbol': symbol,
                'contract_type': contract.get('contractType', ''),
                'status': contract.get('status', ''),
                'current_price': current_price,
                'volume_24h': float(ticker.get('volume24h', 0)),
                'turnover_24h': float(ticker.get('turnover24h', 0)),
                **liquidity_metrics
            }
            liquidity_results.append(liquidity_result)
            
            time.sleep(REQUEST_DELAY)
        
        # Создаем DataFrame для ликвидности
        liquidity_df = pd.DataFrame(liquidity_results).sort_values('max_buy_volume_usdt', ascending=False)
        
        # Рассчитываем арбитражные возможности, если есть perpetual цена
        if perpetual_price:
            for result in liquidity_results:
                symbol = result['symbol']
                
                # Пропускаем perpetual контракт при расчете арбитража
                if symbol != perpetual_symbol:
                    expiry_date = self.parse_expiry_date(symbol, asset)
                    if expiry_date:
                        arbitrage_data = self.calculate_arbitrage_profit(
                            perpetual_price, result['current_price'], expiry_date
                        )
                        if arbitrage_data:
                            arbitrage_result = {
                                'asset': asset,
                                'symbol': symbol,
                                'contract_type': result['contract_type'],
                                'futures_price': result['current_price'],
                                'perpetual_price': perpetual_price,
                                'max_buy_volume_usdt': result['max_buy_volume_usdt'],
                                'max_sell_volume_usdt': result['max_sell_volume_usdt'],
                                'spread_pct': result['spread_pct'],
                                **arbitrage_data
                            }
                            arbitrage_results.append(arbitrage_result)
        
        # Создаем DataFrame для арбитража
        arbitrage_df = pd.DataFrame(arbitrage_results).sort_values('annual_return', ascending=False)
        
        return liquidity_df, arbitrage_df
    
    def analyze_all_assets(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Анализ всех активов"""
        all_liquidity = []
        all_arbitrage = []
        
        assets = ['BTC', 'ETH']
        
        for asset in assets:
            liquidity_df, arbitrage_df = self.analyze_asset_contracts(asset)
            
            if not liquidity_df.empty:
                all_liquidity.append(liquidity_df)
            
            if not arbitrage_df.empty:
                all_arbitrage.append(arbitrage_df)
        
        # Объединяем результаты
        combined_liquidity = pd.concat(all_liquidity, ignore_index=True) if all_liquidity else pd.DataFrame()
        combined_arbitrage = pd.concat(all_arbitrage, ignore_index=True) if all_arbitrage else pd.DataFrame()
        
        # Сортируем объединенные результаты
        if not combined_liquidity.empty:
            combined_liquidity = combined_liquidity.sort_values('max_buy_volume_usdt', ascending=False)
        
        if not combined_arbitrage.empty:
            combined_arbitrage = combined_arbitrage.sort_values('annual_return', ascending=False)
        
        return combined_liquidity, combined_arbitrage

def main():
    analyzer = BTCETHArbitrageAnalyzer()
    
    print("=== BTC & ETH ARBITRAGE & LIQUIDITY ANALYZER ===")
    print("Анализ BTC, ETH арбитражных возможностей и ликвидности")
    print(f"Капитал: ${CAPITAL:,} USDT | Плечо: x{LEVERAGE} | Price Impact: ±{PRICE_IMPACT_PCT}%")
    print("="*80)
    
    # Получаем данные для всех активов
    liquidity_df, arbitrage_df = analyzer.analyze_all_assets()
    
    if liquidity_df.empty:
        print("Не удалось получить данные")
        return
    
    # БЛОК 1: АНАЛИЗ ЛИКВИДНОСТИ
    print("\n" + "="*90)
    print("1. АНАЛИЗ ЛИКВИДНОСТИ КОНТРАКТОВ (ВСЕ АКТИВЫ)")
    print("="*90)
    
    active_df = liquidity_df[
        (liquidity_df['status'] == 'Trading') & 
        (liquidity_df['max_buy_volume_usdt'] > MIN_LIQUIDITY_USD) &
        (liquidity_df['volume_24h'] > 0)
    ].copy()
    
    print(f"\nАктивных контрактов с ликвидностью: {len(active_df)}")
    print(f"{'Asset':<6} {'Symbol':<20} {'Type':<12} {'Price':<12} {'Spread%':<8} {'Max Buy $':<15} {'Max Sell $':<15}")
    print("-" * 100)
    
    for _, row in active_df.head(TOP_CONTRACTS_DISPLAY).iterrows():
        print(f"{row['asset']:<6} "
              f"{row['symbol']:<20} "
              f"{row['contract_type']:<12} "
              f"${row['current_price']:<11.2f} "
              f"{row['spread_pct']:<7.3f} "
              f"${row['max_buy_volume_usdt']:<14,.0f} "
              f"${row['max_sell_volume_usdt']:<14,.0f}")
    
    # БЛОК 2: АРБИТРАЖНЫЙ АНАЛИЗ
    if not arbitrage_df.empty:
        print("\n" + "="*100)
        print("2. АРБИТРАЖНЫЕ ВОЗМОЖНОСТИ (ВСЕ АКТИВЫ)")
        print("="*100)
        
        print(f"Размер каждой позиции: ${analyzer.leveraged_position:,} USDT")
        print(f"Дата расчета: {CURRENT_DATE.strftime('%Y-%m-%d')}")
        
        print(f"\n{'Asset':<6} {'Contract':<20} {'Futures $':<12} {'Diff $':<10} {'Days':<5} {'Profit $':<12} {'Profit %':<8} {'Annual %':<10}")
        print("-" * 95)
        
        for _, row in arbitrage_df.head(15).iterrows():
            print(f"{row['asset']:<6} "
                  f"{row['symbol']:<20} "
                  f"${row['futures_price']:<11.2f} "
                  f"${row['price_difference']:<9.2f} "
                  f"{row['days_to_expiry']:<4d} "
                  f"${row['profit_usdt']:<11.2f} "
                  f"{row['profit_percentage']:<7.2f} "
                  f"{row['annual_return']:<9.2f}")
        
        # Топ-5 арбитражные возможности
        print("\n" + "="*80)
        print("ЛУЧШИЕ АРБИТРАЖНЫЕ ВОЗМОЖНОСТИ (ВСЕ АКТИВЫ):")
        print("="*80)
        
        top_arbitrage = arbitrage_df.head(5)
        for i, (_, row) in enumerate(top_arbitrage.iterrows(), 1):
            # Проверяем достаточность ликвидности
            required_volume = analyzer.leveraged_position
            liquidity_ok = row['max_buy_volume_usdt'] >= required_volume
            liquidity_status = "✅ Достаточно" if liquidity_ok else "⚠️ Недостаточно"
            
            print(f"\n{i}. {row['asset']} - {row['symbol']}")
            print(f"   Прибыль: ${row['profit_usdt']:,.2f} ({row['profit_percentage']:.2f}%)")
            print(f"   Годовая доходность: {row['annual_return']:.2f}%")
            print(f"   Срок: {row['days_to_expiry']} дней")
            print(f"   Ликвидность: {liquidity_status} (${row['max_buy_volume_usdt']:,.0f} доступно)")
            print(f"   Спред: {row['spread_pct']:.3f}%")
            print(f"   Perpetual: ${row['perpetual_price']:.2f} | Futures: ${row['futures_price']:.2f}")
    
    # Сохранение результатов
    if not liquidity_df.empty:
        liquidity_df.to_csv('btc_eth_liquidity_analysis.csv', index=False)
        print(f"\nРезультаты ликвидности сохранены в 'btc_eth_liquidity_analysis.csv'")
    
    if not arbitrage_df.empty:
        arbitrage_df.to_csv('btc_eth_arbitrage_analysis.csv', index=False)
        print(f"Результаты арбитража сохранены в 'btc_eth_arbitrage_analysis.csv'")

if __name__ == "__main__":
    main()

=== BTC & ETH ARBITRAGE & LIQUIDITY ANALYZER ===
Анализ BTC, ETH арбитражных возможностей и ликвидности
Капитал: $50,000 USDT | Плечо: x5 | Price Impact: ±1.0%

--- Анализируем BTC контракты ---
Найдено 15 BTC контрактов, отфильтровано 9
Получаем цену perpetual контракта BTCUSDT...
Цена BTC perpetual (BTCUSDT): $109079.90
Анализируем BTCUSDT (1/9)...
Анализируем BTCUSDT-04JUL25 (2/9)...
Анализируем BTCUSDT-11JUL25 (3/9)...
Анализируем BTCUSDT-18JUL25 (4/9)...
Анализируем BTCUSDT-25JUL25 (5/9)...
Анализируем BTCUSDT-26DEC25 (6/9)...
Анализируем BTCUSDT-26SEP25 (7/9)...
Анализируем BTCUSDT-27MAR26 (8/9)...
Анализируем BTCUSDT-29AUG25 (9/9)...

--- Анализируем ETH контракты ---
Найдено 15 ETH контрактов, отфильтровано 9
Получаем цену perpetual контракта ETHUSDT...
Цена ETH perpetual (ETHUSDT): $2570.43
Анализируем ETHUSDT (1/9)...
Анализируем ETHUSDT-04JUL25 (2/9)...
Анализируем ETHUSDT-11JUL25 (3/9)...
Анализируем ETHUSDT-18JUL25 (4/9)...
Анализируем ETHUSDT-25JUL25 (5/9)...
Анализируем 